# 1) Import the necessary packages

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(color_codes=True)
from collections import Counter,namedtuple,defaultdict,OrderedDict
import warnings
warnings.filterwarnings('ignore')

from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from matplotlib import pyplot

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# 2) Loading the Admission Table

In [3]:
data_AdmissionsCorePopulatedTable=pd.read_csv("/content/drive/My Drive/Colab Notebooks/AdmissionsCorePopulatedTable.txt",sep='\t')
data_AdmissionsCorePopulatedTable.head(10)

,PatientID,AdmissionID,AdmissionStartDate,AdmissionEndDate
0,9380F9E3-1927-42F3-9731-03A74D4E4C6B,5,2011-03-23 00:14:52.237,2011-03-28 01:57:56.467
1,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,1,1974-02-10 00:04:28.357,1974-02-16 21:04:43.333
2,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,2,1991-05-22 15:08:52.743,1991-05-29 12:30:08.737
3,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,3,1995-02-26 08:01:26.477,1995-02-28 16:45:04.407
4,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,4,2005-03-17 04:51:15.747,2005-04-04 04:04:14.160
5,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,5,2008-04-12 23:17:33.217,2008-04-16 03:22:35.737
6,EB1C36B1-0697-4145-95EE-0B0C44943339,1,1995-12-04 16:01:07.700,1995-12-17 22:59:46.223
7,EB1C36B1-0697-4145-95EE-0B0C44943339,2,2001-08-02 05:05:13.547,2001-08-16 02:35:48.323
8,EB1C36B1-0697-4145-95EE-0B0C44943339,3,2009-11-15 09:31:16.750,2009-12-01 06:49:13.053
9,F5F37898-D271-4D68-9C0E-18B1184411C6,1,1989-03-15 16:32:19.170,1989-03-19 07:04:55.640


In [4]:
data_AdmissionsCorePopulatedTable['AdmissionEndDate']=pd.to_datetime(data_AdmissionsCorePopulatedTable['AdmissionEndDate']).dt.date

In [5]:
data_AdmissionsCorePopulatedTable['AdmissionStartDate']=pd.to_datetime(data_AdmissionsCorePopulatedTable['AdmissionStartDate']).dt.date

In [6]:
data_AdmissionsCorePopulatedTable.head(5)

,PatientID,AdmissionID,AdmissionStartDate,AdmissionEndDate
0,9380F9E3-1927-42F3-9731-03A74D4E4C6B,5,2011-03-23,2011-03-28
1,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,1,1974-02-10,1974-02-16
2,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,2,1991-05-22,1991-05-29
3,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,3,1995-02-26,1995-02-28
4,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,4,2005-03-17,2005-04-04


In [7]:
data_AdmissionsCorePopulatedTable['AdmissionDuration']=(data_AdmissionsCorePopulatedTable['AdmissionEndDate']-data_AdmissionsCorePopulatedTable['AdmissionStartDate'])

In [8]:
data_AdmissionsCorePopulatedTable.head(10)

,PatientID,AdmissionID,AdmissionStartDate,AdmissionEndDate,AdmissionDuration
0,9380F9E3-1927-42F3-9731-03A74D4E4C6B,5,2011-03-23,2011-03-28,5 days
1,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,1,1974-02-10,1974-02-16,6 days
2,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,2,1991-05-22,1991-05-29,7 days
3,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,3,1995-02-26,1995-02-28,2 days
4,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,4,2005-03-17,2005-04-04,18 days
5,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,5,2008-04-12,2008-04-16,4 days
6,EB1C36B1-0697-4145-95EE-0B0C44943339,1,1995-12-04,1995-12-17,13 days
7,EB1C36B1-0697-4145-95EE-0B0C44943339,2,2001-08-02,2001-08-16,14 days
8,EB1C36B1-0697-4145-95EE-0B0C44943339,3,2009-11-15,2009-12-01,16 days
9,F5F37898-D271-4D68-9C0E-18B1184411C6,1,1989-03-15,1989-03-19,4 days


In [9]:
data_AdmissionsCorePopulatedTable.shape

(361760, 5)

# 3) Remove the duplicate rows

In [10]:
data_AdmissionsCorePopulatedTable.duplicated(subset=["PatientID","AdmissionID","AdmissionDuration"],keep='first')

0         False
1         False
2         False
3         False
4         False
          ...  
361755    False
361756    False
361757    False
361758    False
361759    False
Length: 361760, dtype: bool

In [11]:
data_AdmissionsCorePopulatedTable.PatientID.count()

361760

In [12]:
data_AdmissionsCorePopulatedTable=data_AdmissionsCorePopulatedTable.drop_duplicates(subset=["PatientID","AdmissionID","AdmissionDuration"],keep='first')

In [13]:
data_AdmissionsCorePopulatedTable.PatientID.count()

361760

In [14]:
data_AdmissionsCorePopulatedTable=data_AdmissionsCorePopulatedTable.drop(columns=['AdmissionStartDate','AdmissionEndDate'],axis=1)
data_AdmissionsCorePopulatedTable.head()

,PatientID,AdmissionID,AdmissionDuration
0,9380F9E3-1927-42F3-9731-03A74D4E4C6B,5,5 days
1,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,1,6 days
2,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,2,7 days
3,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,3,2 days
4,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,4,18 days


In [15]:
data_AdmissionsCorePopulatedTable.isna().sum().to_frame().T

,PatientID,AdmissionID,AdmissionDuration
0,0,0,0


# 5) Loading the Admission Diagnosis Table

In [16]:
data_AdmissionsDiagnosesCorePopulatedTable=pd.read_csv("/content/drive/My Drive/Colab Notebooks/AdmissionsDiagnosesCorePopulatedTable.txt",sep='\t')
data_AdmissionsDiagnosesCorePopulatedTable.head(6)

,PatientID,AdmissionID,PrimaryDiagnosisCode,PrimaryDiagnosisDescription
0,54C6E968-45B3-46B1-A64F-2CE3124F2A65,3,F80.1,Expressive language disorder
1,54C6E968-45B3-46B1-A64F-2CE3124F2A65,4,R04.81,Acute idiopathic pulmonary hemorrhage in infants
2,54C6E968-45B3-46B1-A64F-2CE3124F2A65,5,I36,Nonrheumatic tricuspid valve disorders
3,9DD23357-9BEB-43E4-802D-1AB7ACDD4A3A,1,H16.43,Localized vascularization of cornea
4,9DD23357-9BEB-43E4-802D-1AB7ACDD4A3A,2,M05.161,Rheumatoid lung disease with rheumatoid arthri...
5,9DD23357-9BEB-43E4-802D-1AB7ACDD4A3A,3,M06.011,Rheumatoid arthritis without rheumatoid factor...


# 6) Removing the duplicate rows

In [17]:
data_AdmissionsDiagnosesCorePopulatedTable.duplicated(subset=["PatientID","AdmissionID","PrimaryDiagnosisCode","PrimaryDiagnosisDescription"],keep='first')

0         False
1         False
2         False
3         False
4         False
          ...  
361755    False
361756    False
361757    False
361758    False
361759    False
Length: 361760, dtype: bool

In [18]:
data_AdmissionsDiagnosesCorePopulatedTable=data_AdmissionsDiagnosesCorePopulatedTable.drop_duplicates(subset=["PatientID","AdmissionID","PrimaryDiagnosisCode","PrimaryDiagnosisDescription"],keep='first')

In [19]:
data_AdmissionsDiagnosesCorePopulatedTable.PatientID.count()

361760

In [20]:
data_AdmissionsDiagnosesCorePopulatedTable.isna().sum().to_frame().T

,PatientID,AdmissionID,PrimaryDiagnosisCode,PrimaryDiagnosisDescription
0,0,0,0,0


In [21]:
data_AdmissionsDiagnosesCorePopulatedTable.head(10)

,PatientID,AdmissionID,PrimaryDiagnosisCode,PrimaryDiagnosisDescription
0,54C6E968-45B3-46B1-A64F-2CE3124F2A65,3,F80.1,Expressive language disorder
1,54C6E968-45B3-46B1-A64F-2CE3124F2A65,4,R04.81,Acute idiopathic pulmonary hemorrhage in infants
2,54C6E968-45B3-46B1-A64F-2CE3124F2A65,5,I36,Nonrheumatic tricuspid valve disorders
3,9DD23357-9BEB-43E4-802D-1AB7ACDD4A3A,1,H16.43,Localized vascularization of cornea
4,9DD23357-9BEB-43E4-802D-1AB7ACDD4A3A,2,M05.161,Rheumatoid lung disease with rheumatoid arthri...
5,9DD23357-9BEB-43E4-802D-1AB7ACDD4A3A,3,M06.011,Rheumatoid arthritis without rheumatoid factor...
6,CE969DDB-A168-49C4-9D48-65CFAE878B5F,1,M02.35,"Reiter's disease, hip"
7,CE969DDB-A168-49C4-9D48-65CFAE878B5F,2,C40.31,Malignant neoplasm of short bones of right low...
8,CE969DDB-A168-49C4-9D48-65CFAE878B5F,3,O04.82,Renal failure following (induced) termination ...
9,719278F5-305F-432D-BE0E-76ECE46FD6EA,1,O03.33,Metabolic disorder following incomplete sponta...


# 7) Merging Admission Core table and Admission Diagnosis table to generate Admission Group

In [22]:
data_AdmissionsCorePopulatedTable.dtypes

PatientID                     object
AdmissionID                    int64
AdmissionDuration    timedelta64[ns]
dtype: object

In [23]:
data_AdmissionsDiagnosesCorePopulatedTable.dtypes

PatientID                      object
AdmissionID                     int64
PrimaryDiagnosisCode           object
PrimaryDiagnosisDescription    object
dtype: object

In [24]:
data_AdmissionsCorePopulatedTable.sort_values(by=['PatientID','AdmissionID'])

,PatientID,AdmissionID,AdmissionDuration
320049,0000585C-5C9D-49BD-8F9E-41345464F832,1,10 days
320050,0000585C-5C9D-49BD-8F9E-41345464F832,2,13 days
125111,00005E17-D776-4260-8566-3BF18422A777,1,7 days
125112,00005E17-D776-4260-8566-3BF18422A777,2,5 days
32930,00008B98-12EC-484A-A730-788707EB72C8,1,13 days
...,...,...,...
247473,FFFF554A-83B7-4816-A5D1-B6BBA5A94DCB,2,8 days
247474,FFFF554A-83B7-4816-A5D1-B6BBA5A94DCB,3,13 days
247475,FFFF554A-83B7-4816-A5D1-B6BBA5A94DCB,4,2 days
140311,FFFFA309-DD78-4DEE-8082-921B734062C2,1,11 days


In [25]:
data_AdmissionsDiagnosesCorePopulatedTable.sort_values(by=['PatientID','AdmissionID'],ascending=True)

,PatientID,AdmissionID,PrimaryDiagnosisCode,PrimaryDiagnosisDescription
320049,0000585C-5C9D-49BD-8F9E-41345464F832,1,O03.33,Metabolic disorder following incomplete sponta...
320050,0000585C-5C9D-49BD-8F9E-41345464F832,2,O98.612,"Protozoal diseases complicating pregnancy, sec..."
125111,00005E17-D776-4260-8566-3BF18422A777,1,C05.0,Malignant neoplasm of hard palate
125112,00005E17-D776-4260-8566-3BF18422A777,2,C02.1,Malignant neoplasm of border of tongue
32930,00008B98-12EC-484A-A730-788707EB72C8,1,C93.11,"Chronic myelomonocytic leukemia, in remission"
...,...,...,...,...
247473,FFFF554A-83B7-4816-A5D1-B6BBA5A94DCB,2,M36,Systemic disorders of connective tissue in dis...
247474,FFFF554A-83B7-4816-A5D1-B6BBA5A94DCB,3,M1A.35,"Chronic gout due to renal impairment, hip"
247475,FFFF554A-83B7-4816-A5D1-B6BBA5A94DCB,4,M06.262,"Rheumatoid bursitis, left knee"
140311,FFFFA309-DD78-4DEE-8082-921B734062C2,1,E09.65,Drug or chemical induced diabetes mellitus wit...


In [26]:
data_Admissions_Core_Diagnoses_Merge=data_AdmissionsCorePopulatedTable.merge(data_AdmissionsDiagnosesCorePopulatedTable,how='outer',on=['PatientID','AdmissionID'])
data_Admissions_Core_Diagnoses_Merge.head()

,PatientID,AdmissionID,AdmissionDuration,PrimaryDiagnosisCode,PrimaryDiagnosisDescription
0,9380F9E3-1927-42F3-9731-03A74D4E4C6B,5,5 days,M11.07,"Hydroxyapatite deposition disease, ankle and foot"
1,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,1,6 days,C13.2,Malignant neoplasm of posterior wall of hypoph...
2,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,2,7 days,G30,Alzheimer's disease
3,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,3,2 days,H67.2,"Otitis media in diseases classified elsewhere,..."
4,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,4,18 days,M50.1,Cervical disc disorder with radiculopathy


In [27]:
data_Admissions_Core_Diagnoses_Merge.head(5)

,PatientID,AdmissionID,AdmissionDuration,PrimaryDiagnosisCode,PrimaryDiagnosisDescription
0,9380F9E3-1927-42F3-9731-03A74D4E4C6B,5,5 days,M11.07,"Hydroxyapatite deposition disease, ankle and foot"
1,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,1,6 days,C13.2,Malignant neoplasm of posterior wall of hypoph...
2,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,2,7 days,G30,Alzheimer's disease
3,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,3,2 days,H67.2,"Otitis media in diseases classified elsewhere,..."
4,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,4,18 days,M50.1,Cervical disc disorder with radiculopathy


In [28]:
data_Admissions_Core_Diagnoses_Merge.isna().sum().to_frame().T

,PatientID,AdmissionID,AdmissionDuration,PrimaryDiagnosisCode,PrimaryDiagnosisDescription
0,0,0,0,0,0


# 8) Loading the Patient Table

In [29]:
data_PatientCorePopulatedTable=pd.read_csv("/content/drive/My Drive/Colab Notebooks/PatientCorePopulatedTable.txt",sep="\t")
data_PatientCorePopulatedTable.head(20)

,PatientID,PatientGender,PatientDateOfBirth,PatientRace,PatientMaritalStatus,PatientLanguage,PatientPopulationPercentageBelowPoverty
0,F7CF0FE9-AFCD-49EF-BFB3-E42302FFA0D3,Female,1951-07-10 07:29:47.293,Asian,Single,English,13.70
1,C3935FBC-DBBA-4844-BBE4-A175FA508454,Male,1956-01-27 22:46:39.380,African American,Single,English,15.73
2,1CA33F6F-2E84-4C99-AF6A-D40F7B4DB27F,Male,1972-12-22 10:11:01.867,White,Married,English,7.09
3,81606388-2471-42A4-A6F1-1868AE25CFC3,Male,1984-01-17 00:49:06.903,Asian,Separated,Spanish,2.17
4,E3120DE9-3361-40CF-A618-265C769E75A2,Female,1978-12-21 07:24:08.957,White,Married,English,18.67
5,5C043111-3F94-44BC-A889-97D44ACCC7F6,Female,1974-09-25 18:38:02.440,African American,Married,English,2.57
6,0A10A379-05B1-438B-84A0-18B4D1AE2615,Female,1986-10-23 18:07:05.173,Asian,Married,Icelandic,14.26
7,4BE9DA70-090C-45D9-BB34-C9FE07883BD9,Female,1988-04-07 13:26:59.347,White,Married,English,11.03
8,DED90E65-5B8E-497C-9A17-0679C1DA3B6D,Male,1940-09-29 16:37:18.737,African American,Married,English,11.86
9,67744717-0461-45C5-B85A-D7355EFAF6A1,Female,1967-07-17 12:35:46.713,White,Single,English,5.35


# 9) Removing the duplicate rows

In [30]:
data_PatientCorePopulatedTable.duplicated(subset=['PatientID','PatientGender','PatientMaritalStatus','PatientDateOfBirth'],keep='first')

0        False
1        False
2        False
3        False
4        False
         ...  
99995    False
99996    False
99997    False
99998    False
99999    False
Length: 100000, dtype: bool

In [31]:
data_PatientCorePopulatedTable.PatientID.count()

100000

# 10) Dropping the columns which are not required

In [32]:
data_PatientCorePopulatedTable=data_PatientCorePopulatedTable.drop(columns=['PatientLanguage','PatientPopulationPercentageBelowPoverty'],axis=1)
data_PatientCorePopulatedTable.head()

,PatientID,PatientGender,PatientDateOfBirth,PatientRace,PatientMaritalStatus
0,F7CF0FE9-AFCD-49EF-BFB3-E42302FFA0D3,Female,1951-07-10 07:29:47.293,Asian,Single
1,C3935FBC-DBBA-4844-BBE4-A175FA508454,Male,1956-01-27 22:46:39.380,African American,Single
2,1CA33F6F-2E84-4C99-AF6A-D40F7B4DB27F,Male,1972-12-22 10:11:01.867,White,Married
3,81606388-2471-42A4-A6F1-1868AE25CFC3,Male,1984-01-17 00:49:06.903,Asian,Separated
4,E3120DE9-3361-40CF-A618-265C769E75A2,Female,1978-12-21 07:24:08.957,White,Married


In [33]:
data_PatientCorePopulatedTable.PatientID.count()

100000

In [34]:
data_PatientCorePopulatedTable.isna().sum().to_frame().T

,PatientID,PatientGender,PatientDateOfBirth,PatientRace,PatientMaritalStatus
0,0,0,0,0,0


In [35]:
data_PatientCorePopulatedTable.sort_values(by=['PatientID'],ascending=True)

,PatientID,PatientGender,PatientDateOfBirth,PatientRace,PatientMaritalStatus
88450,0000585C-5C9D-49BD-8F9E-41345464F832,Female,1984-04-28 23:40:27.293,White,Single
34632,00005E17-D776-4260-8566-3BF18422A777,Female,1924-08-02 09:42:39.840,White,Unknown
9238,00008B98-12EC-484A-A730-788707EB72C8,Male,1948-06-11 09:06:23.503,White,Divorced
78349,0001452D-730F-45E4-A896-3F95DB98849D,Female,1948-07-28 02:33:01.517,White,Married
64581,0001E6FA-A3D7-4569-A0B3-7E9865EFF6B6,Male,1987-02-07 02:56:53.157,Asian,Single
...,...,...,...,...,...
79794,FFFDE252-E3DC-4909-AEAD-EFC88A90EECE,Male,1965-07-28 18:37:03.817,Unknown,Single
12007,FFFE2CE0-297E-4134-8821-D6AA79468A9E,Female,1965-02-06 21:47:31.713,White,Married
76666,FFFE58EA-FE3E-4631-85F7-CFAB1FCAEE0A,Female,1963-11-29 20:01:08.360,White,Married
68370,FFFF554A-83B7-4816-A5D1-B6BBA5A94DCB,Female,1966-02-01 18:25:14.407,White,Single


# 11) Merging Patient table with Admission group to generate Patient group

In [36]:
data_Admissions_Core_Diagnoses_Merge['PatientID'].count()

361760

In [37]:
data_PatientCorePopulatedTable['PatientID'].count()

100000

In [38]:
data_Patients_Core_Admission_Core_Diagnoses_Merge=data_PatientCorePopulatedTable.merge(data_Admissions_Core_Diagnoses_Merge,indicator=False,how='outer')
data_Patients_Core_Admission_Core_Diagnoses_Merge.shape

(361760, 9)

In [39]:
data_PatientCorePopulatedTable.isna().sum().to_frame().T

,PatientID,PatientGender,PatientDateOfBirth,PatientRace,PatientMaritalStatus
0,0,0,0,0,0


In [40]:
data_Patients_Core_Admission_Core_Diagnoses_Merge.head(20)

,PatientID,PatientGender,PatientDateOfBirth,PatientRace,PatientMaritalStatus,AdmissionID,AdmissionDuration,PrimaryDiagnosisCode,PrimaryDiagnosisDescription
0,F7CF0FE9-AFCD-49EF-BFB3-E42302FFA0D3,Female,1951-07-10 07:29:47.293,Asian,Single,1,10 days,O10.13,Pre-existing hypertensive heart disease compli...
1,F7CF0FE9-AFCD-49EF-BFB3-E42302FFA0D3,Female,1951-07-10 07:29:47.293,Asian,Single,2,12 days,E74.1,Disorders of fructose metabolism
2,F7CF0FE9-AFCD-49EF-BFB3-E42302FFA0D3,Female,1951-07-10 07:29:47.293,Asian,Single,3,16 days,C91.02,"Acute lymphoblastic leukemia, in relapse"
3,C3935FBC-DBBA-4844-BBE4-A175FA508454,Male,1956-01-27 22:46:39.380,African American,Single,1,12 days,M10.34,"Gout due to renal impairment, hand"
4,C3935FBC-DBBA-4844-BBE4-A175FA508454,Male,1956-01-27 22:46:39.380,African American,Single,2,17 days,F44.4,Conversion disorder with motor symptom or deficit
5,C3935FBC-DBBA-4844-BBE4-A175FA508454,Male,1956-01-27 22:46:39.380,African American,Single,3,16 days,I39,Endocarditis and heart valve disorders in dise...
6,C3935FBC-DBBA-4844-BBE4-A175FA508454,Male,1956-01-27 22:46:39.380,African American,Single,4,18 days,C92.00,"Acute myeloblastic leukemia, not having achiev..."
7,C3935FBC-DBBA-4844-BBE4-A175FA508454,Male,1956-01-27 22:46:39.380,African American,Single,5,16 days,O99.613,Diseases of the digestive system complicating ...
8,1CA33F6F-2E84-4C99-AF6A-D40F7B4DB27F,Male,1972-12-22 10:11:01.867,White,Married,1,5 days,C90,Multiple myeloma and malignant plasma cell neo...
9,1CA33F6F-2E84-4C99-AF6A-D40F7B4DB27F,Male,1972-12-22 10:11:01.867,White,Married,2,6 days,F23,Brief psychotic disorder


In [41]:
data_Patients_Core_Admission_Core_Diagnoses_Merge.PatientID.count()

361760

In [42]:
data_Patients_Core_Admission_Core_Diagnoses_Merge.isna().sum().to_frame().T

,PatientID,PatientGender,PatientDateOfBirth,PatientRace,PatientMaritalStatus,AdmissionID,AdmissionDuration,PrimaryDiagnosisCode,PrimaryDiagnosisDescription
0,0,0,0,0,0,0,0,0,0


# 12) Loading the Labs Table

In [45]:
import random

datafile='/content/drive/My Drive/Colab Notebooks/LabsCorePopulatedTable.txt'
n=sum(1 for line in open(datafile))-1
s=n//10
skip=sorted(random.sample(range(1, n+1), n-s))
data_LabsCorePopulatedTable=pd.read_csv(datafile,sep='\t',skiprows=skip)
data_LabsCorePopulatedTable.head()

OSError: ignored

In [ ]:
data_LabsCorePopulatedTable.PatientID.count()

In [ ]:
data_LabsCorePopulatedTable.isna().sum().to_frame().T

In [ ]:
data_LabsCorePopulatedTable=data_LabsCorePopulatedTable.drop(columns=['LabUnits','LabDateTime'],axis=1)
data_LabsCorePopulatedTable.head()

In [ ]:
Counter(data_LabsCorePopulatedTable.LabName) 

#13) Use one-hot encoding to find out the different Lab Names

In [ ]:
from sklearn.preprocessing import OneHotEncoder,LabelEncoder 